In [ ]:
!pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 16.3 MB/s eta 0:00:00


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import pymupdf4llm
import re
import os
import zipfile
import gradio as gr

In [ ]:
def extract_markdown_elements(lines):
    """
    Extract both content sections and tables from markdown lines.

    Args:
        lines (list): List of markdown lines

    Returns:
        tuple: (content_sections, tables)
            - content_sections: List of (heading, level, content)
            - tables: List of (associated_heading, table_content)
    """
    # Regular expressions
    heading_pattern = r'^(#+)\s+(.*)$'

    # Initialize containers
    content_sections = []
    tables = []

    # State tracking variables
    current_heading = None
    current_content = []
    current_level = 0

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Table detection
        if line.startswith('|'):
            table_content = []
            # Collect entire table
            while i < len(lines) and lines[i].strip().startswith('|'):
                table_content.append(lines[i])
                i += 1

            # Check if it's a valid table (at least header, separator, and one data row)
            if len(table_content) > 2:
                table_text = '\n'.join(table_content)
                tables.append((current_heading, table_text))
            continue

        # Heading detection
        heading_match = re.match(heading_pattern, line, re.MULTILINE)
        if heading_match:
            # Save previous section if it exists
            if current_heading and current_content:
                content_sections.append((current_heading, current_level, '\n'.join(current_content)))
                current_content = []

            # Start new section
            current_heading = heading_match.group(2)
            current_level = len(heading_match.group(1))
            i += 1
            continue

        # Collect content
        if line:
            current_content.append(line)

        i += 1

    # Save last section if exists
    if current_heading and current_content:
        content_sections.append((current_heading, current_level, '\n'.join(current_content)))

    return content_sections, tables


In [ ]:

def organize_markdown_content(input_pdf_path, output_dir):
    """
    Convert PDF to markdown, extract tables, headings, and organize content.

    Args:
        input_pdf_path (str): Path to input PDF file
        output_dir (str): Directory to save organized markdown files
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Convert PDF to markdown
    md_text = pymupdf4llm.to_markdown(input_pdf_path)

    # Split content into lines
    lines = md_text.split('\n')

    # Extract content sections and tables
    content_sections, tables = extract_markdown_elements(lines)

    # Write document content
    with open(os.path.join(output_dir, 'document_content.md'), 'w', encoding='utf-8') as f:
        for heading, level, content in content_sections:
            f.write('#' * level + ' ' + heading + '\n\n')
            f.write(content + '\n\n')

    # Write tables
    if tables:
        with open(os.path.join(output_dir, 'tables.md'), 'w', encoding='utf-8') as f:
            f.write('# Tables\n\n')
            for heading, table in tables:
                if heading:
                    f.write(f'## From section: {heading}\n\n')
                f.write(table + '\n\n')

    # Create summary file
    with open(os.path.join(output_dir, 'summary.md'), 'w', encoding='utf-8') as f:
        f.write('# Document Summary\n\n')
        f.write(f'- Total tables: {len(tables)}\n')
        f.write(f'- Total content sections: {len(content_sections)}\n')

In [ ]:
def process_pdf(pdf_file):
    # Define the output directory and paths
    output_dir = "organized_content"
    zip_filename = "organized_content.zip"

    # Run the PDF processing function
    organize_markdown_content(pdf_file, output_dir)

    # Zip the output directory
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, _, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, output_dir))

    return zip_filename

# Gradio UI
def gradio_interface(pdf_file):
    zip_filepath = process_pdf(pdf_file.name)
    return zip_filepath

# Define Gradio input and output
gr.Interface(
    fn=gradio_interface,
    inputs=gr.File(label="Upload PDF File"),
    outputs=gr.File(label="Download ZIP File"),
    title="PDF to Organized Markdown ZIP Converter",
    description="Upload a PDF file, and download a ZIP file with organized markdown content."
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6593182b47e0f050c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
